#### imports

#**TO-DOs:**
- Test classifier chain
    - ~~full data~~
    - selected RF data
- Feature selection (tentative)
    - PCA
    - lasso
    - (?)RFECV
- ~~(?)EDA: Cramér's V correlation matrix~~

In [1]:
import pandas as pd
import numpy as np
import random
import os

import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno
#from sklearn.manifold import TSNE

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain

from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV

from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.metrics import jaccard_score

In [2]:
df = pd.read_csv("/content/enzymes.csv")
df=df.drop("Unnamed: 0",axis="columns")
print(df.shape,"\n\n")
df.head(10)

(1039, 1228) 




,CIDs,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,F510,F511,F512,EC1_EC2_EC3_EC4_EC5_EC6,EC1,EC2,EC3,EC4,EC5,EC6
0,C00009,8.881944,-4.638889,8.881944,4.638889,0.341891,97.994,94.970,97.976895,32,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
1,C00013,9.626875,-5.050347,9.626875,2.958333,0.407078,177.973,173.941,177.943226,56,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
2,C00014,0.000000,0.000000,0.000000,0.000000,0.397555,17.031,14.007,17.026549,8,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
3,C00017,10.860741,-0.931481,10.860741,0.215832,0.407584,130.103,124.055,130.037842,50,...,0,0,0,0_1_1_0_0_0,0,1,1,0,0,0
4,C00022,9.543981,-1.379630,9.543981,0.824074,0.445928,88.062,84.030,88.016044,34,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
5,C00024,12.612956,-5.557256,12.612956,0.030983,0.058692,809.578,771.274,809.125774,288,...,0,0,0,1_1_1_0_0_1,1,1,1,0,0,1
6,C00025,9.993880,-1.165509,9.993880,0.023148,0.485976,147.130,138.058,147.053158,58,...,0,0,1,1_1_1_1_1_1,1,1,1,1,1,1
7,C00026,10.184398,-1.582176,10.184398,0.425463,0.521625,146.098,140.050,146.021523,56,...,0,0,0,1_1_0_1_0_1,1,1,0,1,0,1
8,C00027,6.000000,6.000000,6.000000,6.000000,0.292487,34.014,31.998,34.005479,14,...,0,0,0,1_0_0_1_0_0,1,0,0,1,0,0
9,C00029,12.154474,-5.592374,12.154474,0.722789,0.126671,566.302,542.110,566.055021,206,...,0,0,0,1_1_1_1_1_0,1,1,1,1,1,0


In [3]:
x = ! ls *f*

x = x[0].split()
x

['df_desc.csv', 'df_ecfp.csv', 'df_fcfp.csv', 'rf_top10s.csv']

In [4]:
df_desc = pd.read_csv("/content/df_desc.csv", index_col=0)
df_ecfp = pd.read_csv("/content/df_ecfp.csv", index_col=0)
df_fcfp = pd.read_csv("/content/df_fcfp.csv", index_col=0)
rf_top10s = pd.read_csv("/content/rf_top10s.csv", index_col=0)

In [11]:
df_desc.head()

,CIDs,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_thiophene,fr_unbrch_alkane,fr_urea,EC1,EC2,EC3,EC4,EC5,EC6,EC1_EC2_EC3_EC4_EC5_EC6
0,C00009,8.881944,-4.638889,8.881944,4.638889,0.341891,97.994,94.970,97.976895,32,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
1,C00013,9.626875,-5.050347,9.626875,2.958333,0.407078,177.973,173.941,177.943226,56,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
2,C00014,0.000000,0.000000,0.000000,0.000000,0.397555,17.031,14.007,17.026549,8,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
3,C00017,10.860741,-0.931481,10.860741,0.215832,0.407584,130.103,124.055,130.037842,50,...,0,0,0,0,1,1,0,0,0,0_1_1_0_0_0
4,C00022,9.543981,-1.379630,9.543981,0.824074,0.445928,88.062,84.030,88.016044,34,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1


In [12]:
df_ecfp.head()

,CIDs,M1,M2,M3,M4,M5,M6,M7,M8,M9,...,M510,M511,M512,EC1,EC2,EC3,EC4,EC5,EC6,EC1_EC2_EC3_EC4_EC5_EC6
0,C00009,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
1,C00013,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
2,C00014,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1
3,C00017,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0_1_1_0_0_0
4,C00022,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,1,1_1_1_1_0_1


In [13]:
df_fcfp.head()

,CIDs,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F510,F511,F512,EC1_EC2_EC3_EC4_EC5_EC6,EC1,EC2,EC3,EC4,EC5,EC6
0,C00009,1,1,1,0,0,0,0,0,0,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
1,C00013,1,1,1,0,0,0,0,0,0,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
2,C00014,0,0,0,1,0,0,0,0,0,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1
3,C00017,1,1,1,0,0,0,0,0,0,...,0,0,0,0_1_1_0_0_0,0,1,1,0,0,0
4,C00022,1,1,1,0,0,0,0,0,0,...,0,0,0,1_1_1_1_0_1,1,1,1,1,0,1


In [24]:
len(rf_top10s["col"].unique())

72

In [25]:
list((df.columns[-6:]))

['EC1', 'EC2', 'EC3', 'EC4', 'EC5', 'EC6']

In [30]:
top10s=list(rf_top10s["col"].unique())
top10s.extend(list((df.columns[-6:])))
len(top10s)

78

In [31]:
top10s.insert(0,"CIDs")

In [32]:
len(top10s)

79

In [33]:
df_top10s = df[top10s]

df_top10s.head(10)

,CIDs,MolLogP,MinEStateIndex,TPSA,NumHeteroatoms,MinAbsEStateIndex,FpDensityMorgan1,FpDensityMorgan2,VSA_EState9,Kappa2,...,M293,M357,M506,M338,EC1,EC2,EC3,EC4,EC5,EC6
0,C00009,-0.9286,-4.638889,77.76,5,4.638889,1.200000,1.200000,16.923611,1.064830,...,0,0,0,0,1,1,1,1,0,1
1,C00013,-0.8116,-5.050347,124.29,9,2.958333,0.888889,1.111111,20.899028,2.476089,...,0,0,0,0,1,1,1,1,0,1
2,C00014,0.1620,0.000000,35.00,1,0.000000,1.000000,1.000000,0.000000,-27.040000,...,0,0,0,0,1,1,1,1,0,1
3,C00017,-2.1033,-0.931481,92.42,7,0.215832,1.777778,2.333333,35.105740,1.696130,...,0,0,0,0,0,1,1,0,0,0
4,C00022,-0.3400,-1.379630,54.37,3,0.824074,1.500000,1.666667,25.333333,1.545017,...,0,0,1,0,1,1,1,1,0,1
5,C00024,-1.3154,-5.557256,363.63,28,0.030983,1.078431,1.745098,99.412210,17.589923,...,0,1,1,1,1,1,1,0,0,1
6,C00025,-0.7369,-1.165509,100.62,5,0.023148,1.400000,1.900000,37.666667,3.837557,...,0,0,0,0,1,1,1,1,1,1
7,C00026,-0.4951,-1.582176,91.67,5,0.425463,1.000000,1.500000,41.000000,3.600317,...,0,0,0,0,1,1,0,1,0,1
8,C00027,0.0174,6.000000,40.46,2,6.000000,1.000000,1.000000,12.000000,0.920000,...,0,0,0,0,1,0,0,1,0,0
9,C00029,-4.7937,-5.592374,296.99,21,0.722789,0.916667,1.555556,83.938867,11.159016,...,0,1,0,1,1,1,1,1,1,0


In [34]:
df_top10s.to_csv("df_top10s.csv")

### Cramér's V (postponed)

In [6]:
dataset = df.copy()

###########CHATGPT######################
cols = dataset.columns
cramers_results = pd.DataFrame(index=cols, columns=cols)

for c1 in cols:
    for c2 in cols:
        confusion_matrix = pd.crosstab(dataset[c1], dataset[c2])
        chi2 = chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        r, k = confusion_matrix.shape
        cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))

cramers_results = cramers_results.astype(float)
#########################################


plt.figure(figsize=(10,8))

ax = sns.heatmap(cramers_results, annot=True, cmap="flare")
shortened = [ "chem_index","parity","Diet","forage_mode","substrate","Activity_time","Limbs","kinesis","Major_Realm","Family" ]
ax.set_yticklabels(shortened[::1])#shorten Y labels
ax.set_xticklabels(shortened[::1])#shorten X labels

plt.title("Cramér’s V (Correlation) Heatmap")
plt.tight_layout()
plt.savefig("cramersv_necessarydf_heatmap.png")
plt.show()

/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-2016381379.py:13: RuntimeWarning: invalid value encountered in divide
  cramers_results.loc[c1, c2] = np.sqrt(chi2 / (n * (min(k - 1, r - 1))))
/tmp/ipython-input-201

KeyboardInterrupt: 

### ClassifierChain (w/full data)

Jaccard Score --> metric for multilabel classification (should be used in addition to precision/f1?)

In [71]:
X = df[df.columns[1:-7]]
X=X.drop("Ipc",axis="columns")
X

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,F503,F504,F505,F506,F507,F508,F509,F510,F511,F512
0,8.881944,-4.638889,8.881944,4.638889,0.341891,97.994,94.970,97.976895,32,0,...,0,0,1,0,0,0,0,0,0,0
1,9.626875,-5.050347,9.626875,2.958333,0.407078,177.973,173.941,177.943226,56,0,...,0,0,1,0,0,0,0,0,0,0
2,0.000000,0.000000,0.000000,0.000000,0.397555,17.031,14.007,17.026549,8,0,...,0,0,0,0,0,0,0,0,0,0
3,10.860741,-0.931481,10.860741,0.215832,0.407584,130.103,124.055,130.037842,50,0,...,0,0,1,0,0,0,1,0,0,0
4,9.543981,-1.379630,9.543981,0.824074,0.445928,88.062,84.030,88.016044,34,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,4.750000,0.250000,4.750000,0.250000,0.371490,16.023,14.007,16.018724,7,0,...,0,0,0,0,0,0,0,0,0,0
1035,13.516859,-0.727746,13.516859,0.138810,0.536654,405.458,382.274,405.180090,154,0,...,1,0,1,0,0,1,0,0,0,0
1036,13.606147,-2.044178,13.606147,0.276604,0.060282,831.693,794.397,831.197041,316,0,...,0,0,1,1,0,0,0,0,0,0
1037,13.774221,-2.149645,13.774221,0.322613,0.051119,993.834,946.458,993.249864,380,0,...,0,0,1,1,0,0,0,0,0,0


In [72]:
Y=df[df.columns[-6:]]
Y

,EC1,EC2,EC3,EC4,EC5,EC6
0,1,1,1,1,0,1
1,1,1,1,1,0,1
2,1,1,1,1,0,1
3,0,1,1,0,0,0
4,1,1,1,1,0,1
...,...,...,...,...,...,...
1034,0,1,0,0,0,1
1035,0,1,0,0,0,0
1036,0,1,0,0,0,0
1037,0,1,1,0,0,0


In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1202)

In [74]:
metrics_df = pd.DataFrame(data=["train_jacc","test_jacc","precision","recall","f1"],columns=["metrics"])
metrics_df

,metrics
0,train_jacc
1,test_jacc
2,precision
3,recall
4,f1


In [75]:
base_rf = RandomForestClassifier(max_depth=10, random_state=1202)
chain = ClassifierChain(base_rf, order='random', random_state=1202)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_RF"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.8379061371841154
Test Jaccard:  0.43637820512820513


,metrics,CC_RF
0,train_jacc,0.837906
1,test_jacc,0.436378
2,precision,0.611878
3,recall,0.532828
4,f1,0.533459


#### try same thing with dt

In [76]:
base_rf = DecisionTreeClassifier(max_depth=12, random_state=1202)
chain = ClassifierChain(base_rf, order='random', random_state=1202)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")


metrics_df["CC_DT"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.8514039310068191
Test Jaccard:  0.3935096153846154


,metrics,CC_RF,CC_DT
0,train_jacc,0.837906,0.851404
1,test_jacc,0.436378,0.393510
2,precision,0.611878,0.525508
3,recall,0.532828,0.560606
4,f1,0.533459,0.539986


#### knn

##### STANDARDIZE DATA

In [77]:
scaler = StandardScaler()

#isolate & skip over binary cols
binary_cols = X.columns[X.nunique() == 2]
cont_cols = X.columns[X.nunique() > 2]

scaler = StandardScaler()
X[cont_cols] = scaler.fit_transform(X[cont_cols])

In [78]:
base_rf = KNeighborsClassifier(n_neighbors=5)
chain = ClassifierChain(base_rf, order='random', random_state=1202)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_KNN"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.545367027677497
Test Jaccard:  0.3715544871794872


,metrics,CC_RF,CC_DT,CC_KNN
0,train_jacc,0.837906,0.851404,0.545367
1,test_jacc,0.436378,0.393510,0.371554
2,precision,0.611878,0.525508,0.482462
3,recall,0.532828,0.560606,0.497475
4,f1,0.533459,0.539986,0.483438


#### SVM

In [79]:
base_rf = SVC(random_state=1202)
chain = ClassifierChain(base_rf, order='random', random_state=1202)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
#y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_SVM"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.4377055756117128
Test Jaccard:  0.43838141025641025


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM
0,train_jacc,0.837906,0.851404,0.545367,0.437706
1,test_jacc,0.436378,0.393510,0.371554,0.438381
2,precision,0.611878,0.525508,0.482462,0.391440
3,recall,0.532828,0.560606,0.497475,0.530303
4,f1,0.533459,0.539986,0.483438,0.442092


Heavy overfitting in tree-based models, no overfitting on SVM?
- RF have significantly fewer false negatives

In [80]:
#metrics_df.to_csv("CC_metrics.csv")

### ClassifierChain (w/ top10s data)
- Stacked meta classifier

In [81]:
df_desc = pd.read_csv("/content/df_desc.csv", index_col=0)
df_ecfp = pd.read_csv("/content/df_ecfp.csv", index_col=0)
df_fcfp = pd.read_csv("/content/df_fcfp.csv", index_col=0)
df_top10s = pd.read_csv("/content/df_top10s.csv",index_col=0)
#metrics_df = pd.read_csv("/content/CC_metrics.csv", index_col=0)

In [82]:
#metrics_df = pd.DataFrame(data=["train_jacc","test_jaccard","precision","recall","f1"],columns=["metrics"])
metrics_df

,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM
0,train_jacc,0.837906,0.851404,0.545367,0.437706
1,test_jacc,0.436378,0.393510,0.371554,0.438381
2,precision,0.611878,0.525508,0.482462,0.391440
3,recall,0.532828,0.560606,0.497475,0.530303
4,f1,0.533459,0.539986,0.483438,0.442092


    Rerunning previous models to record train/test jaccard

#### train/test split

In [83]:
df_top10s.head(10)

,CIDs,MolLogP,MinEStateIndex,TPSA,NumHeteroatoms,MinAbsEStateIndex,FpDensityMorgan1,FpDensityMorgan2,VSA_EState9,Kappa2,...,M293,M357,M506,M338,EC1,EC2,EC3,EC4,EC5,EC6
0,C00009,-0.9286,-4.638889,77.76,5,4.638889,1.200000,1.200000,16.923611,1.064830,...,0,0,0,0,1,1,1,1,0,1
1,C00013,-0.8116,-5.050347,124.29,9,2.958333,0.888889,1.111111,20.899028,2.476089,...,0,0,0,0,1,1,1,1,0,1
2,C00014,0.1620,0.000000,35.00,1,0.000000,1.000000,1.000000,0.000000,-27.040000,...,0,0,0,0,1,1,1,1,0,1
3,C00017,-2.1033,-0.931481,92.42,7,0.215832,1.777778,2.333333,35.105740,1.696130,...,0,0,0,0,0,1,1,0,0,0
4,C00022,-0.3400,-1.379630,54.37,3,0.824074,1.500000,1.666667,25.333333,1.545017,...,0,0,1,0,1,1,1,1,0,1
5,C00024,-1.3154,-5.557256,363.63,28,0.030983,1.078431,1.745098,99.412210,17.589923,...,0,1,1,1,1,1,1,0,0,1
6,C00025,-0.7369,-1.165509,100.62,5,0.023148,1.400000,1.900000,37.666667,3.837557,...,0,0,0,0,1,1,1,1,1,1
7,C00026,-0.4951,-1.582176,91.67,5,0.425463,1.000000,1.500000,41.000000,3.600317,...,0,0,0,0,1,1,0,1,0,1
8,C00027,0.0174,6.000000,40.46,2,6.000000,1.000000,1.000000,12.000000,0.920000,...,0,0,0,0,1,0,0,1,0,0
9,C00029,-4.7937,-5.592374,296.99,21,0.722789,0.916667,1.555556,83.938867,11.159016,...,0,1,0,1,1,1,1,1,1,0


In [84]:
X = df_top10s[df_top10s.columns[1:-6]]
X.head()

,MolLogP,MinEStateIndex,TPSA,NumHeteroatoms,MinAbsEStateIndex,FpDensityMorgan1,FpDensityMorgan2,VSA_EState9,Kappa2,qed,...,M17,M379,M284,M382,M202,M148,M293,M357,M506,M338
0,-0.9286,-4.638889,77.76,5,4.638889,1.200000,1.200000,16.923611,1.064830,0.341891,...,0,0,0,0,0,0,0,0,0,0
1,-0.8116,-5.050347,124.29,9,2.958333,0.888889,1.111111,20.899028,2.476089,0.407078,...,0,0,1,0,0,0,0,0,0,0
2,0.1620,0.000000,35.00,1,0.000000,1.000000,1.000000,0.000000,-27.040000,0.397555,...,0,0,0,0,0,0,0,0,0,0
3,-2.1033,-0.931481,92.42,7,0.215832,1.777778,2.333333,35.105740,1.696130,0.407584,...,0,0,0,1,0,1,0,0,0,0
4,-0.3400,-1.379630,54.37,3,0.824074,1.500000,1.666667,25.333333,1.545017,0.445928,...,0,0,0,1,0,0,0,0,1,0


In [85]:
Y = df_top10s[df_top10s.columns[-6:]]
Y

,EC1,EC2,EC3,EC4,EC5,EC6
0,1,1,1,1,0,1
1,1,1,1,1,0,1
2,1,1,1,1,0,1
3,0,1,1,0,0,0
4,1,1,1,1,0,1
...,...,...,...,...,...,...
1034,0,1,0,0,0,1
1035,0,1,0,0,0,0
1036,0,1,0,0,0,0
1037,0,1,1,0,0,0


In [86]:
X_train

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,F503,F504,F505,F506,F507,F508,F509,F510,F511,F512
346,12.345724,-1.316641,12.345724,0.029219,0.165562,472.462,448.270,472.181880,180,0,...,0,0,1,0,0,0,0,0,0,0
65,10.140139,-4.564715,10.140139,0.763117,0.450689,170.057,163.001,169.998025,60,0,...,0,0,1,1,0,0,0,0,0,0
703,13.410067,-5.891287,13.410067,0.244521,0.026498,1018.038,940.422,1017.457565,394,0,...,0,1,1,1,0,0,0,1,0,0
154,11.611963,-4.640984,11.611963,0.061276,0.489914,307.199,293.087,307.056936,112,0,...,0,1,1,1,0,0,0,0,0,0
860,9.252350,0.171585,9.252350,0.171585,0.693280,164.204,152.108,164.083730,64,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,12.731054,-5.584752,12.731054,0.015803,0.036837,987.853,931.405,987.261539,360,0,...,0,0,1,1,0,0,0,0,0,0
449,10.220616,-4.544514,10.220616,0.162292,0.460188,228.137,215.033,228.039889,84,0,...,0,0,1,1,0,0,0,0,0,0
862,9.533231,-0.054207,9.533231,0.054207,0.864982,270.284,256.172,270.089209,102,0,...,0,0,1,0,0,0,1,0,0,0
891,10.399857,-4.689671,10.399857,0.649877,0.287306,259.151,245.039,259.045703,96,0,...,0,0,1,1,0,0,0,0,0,0


In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1203)

In [88]:
X_train

,MolLogP,MinEStateIndex,TPSA,NumHeteroatoms,MinAbsEStateIndex,FpDensityMorgan1,FpDensityMorgan2,VSA_EState9,Kappa2,qed,...,M17,M379,M284,M382,M202,M148,M293,M357,M506,M338
871,-0.0118,-3.824531,74.60,6,0.245648,1.545455,2.090909,10.518255,4.971069,0.482008,...,0,0,1,0,0,0,0,0,0,0
747,12.7719,0.289811,0.00,0,0.289811,0.650000,1.125000,74.083333,19.041322,0.162107,...,0,0,0,0,0,0,0,1,1,0
695,2.0418,-0.379907,37.30,4,0.111042,1.727273,2.454545,26.778866,5.607504,0.627628,...,0,0,0,1,0,0,1,0,0,0
937,6.7480,0.172567,17.07,3,0.172567,0.850000,1.300000,36.791810,15.154614,0.245549,...,0,0,0,1,0,0,0,0,0,0
680,-1.3184,-5.575941,383.86,29,0.025648,1.036364,1.690909,110.002501,19.310081,0.051367,...,0,1,1,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,-0.1589,0.051936,67.15,4,0.051936,1.153846,1.769231,30.166667,9.477796,0.462587,...,0,0,0,1,0,1,0,0,1,0
405,-2.4166,-5.384346,312.47,23,0.046389,0.977273,1.613636,92.291562,12.410403,0.078024,...,0,1,0,1,0,1,0,1,0,1
46,0.2052,0.750000,17.07,1,0.750000,2.000000,2.000000,11.000000,1.670000,0.355008,...,0,0,0,0,0,0,0,0,0,0
803,2.2499,-1.241554,74.60,4,0.001505,1.421053,2.000000,52.666667,5.131878,0.605332,...,0,0,0,1,0,1,0,1,1,1


#### RF

In [89]:
base_rf = RandomForestClassifier(max_depth=10, random_state=1203)
chain = ClassifierChain(base_rf, order='random', random_state=1203)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}\n")


metrics_df["CC_topRF"] = [train_jaccard, test_jaccard, #jaccard score
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.9062976333734455
Test Jaccard:  0.4181089743589744



,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM,CC_topRF
0,train_jacc,0.837906,0.851404,0.545367,0.437706,0.906298
1,test_jacc,0.436378,0.393510,0.371554,0.438381,0.418109
2,precision,0.611878,0.525508,0.482462,0.391440,0.568360
3,recall,0.532828,0.560606,0.497475,0.530303,0.491525
4,f1,0.533459,0.539986,0.483438,0.442092,0.503232


Accuracy is pass/fail so is slightly misleading

Jaccard score naturally handles multilabel classification



Still overfitting

#### DT

In [90]:
base_rf = DecisionTreeClassifier(max_depth=12, random_state=1203)
chain = ClassifierChain(base_rf, order='random', random_state=1203)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_topDT"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.8914961893301243
Test Jaccard:  0.38653846153846155


,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM,CC_topRF,CC_topDT
0,train_jacc,0.837906,0.851404,0.545367,0.437706,0.906298,0.891496
1,test_jacc,0.436378,0.393510,0.371554,0.438381,0.418109,0.386538
2,precision,0.611878,0.525508,0.482462,0.391440,0.568360,0.528515
3,recall,0.532828,0.560606,0.497475,0.530303,0.491525,0.515738
4,f1,0.533459,0.539986,0.483438,0.442092,0.503232,0.517810


#### KNN

##### STANDARDIZE DATA

- better for KNN & SVM

In [91]:
scaler = StandardScaler()

#isolate & skip over binary cols
binary_cols = X.columns[X.nunique() == 2]
cont_cols = X.columns[X.nunique() > 2]

scaler = StandardScaler()
X[cont_cols] = scaler.fit_transform(X[cont_cols])

/tmp/ipython-input-1655540887.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cont_cols] = scaler.fit_transform(X[cont_cols])


In [92]:
base_rf = KNeighborsClassifier(n_neighbors=5)
chain = ClassifierChain(base_rf, order='random', random_state=1203)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_topKNN"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.5323706377858002
Test Jaccard:  0.358573717948718


,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM,CC_topRF,CC_topDT,CC_topKNN
0,train_jacc,0.837906,0.851404,0.545367,0.437706,0.906298,0.891496,0.532371
1,test_jacc,0.436378,0.393510,0.371554,0.438381,0.418109,0.386538,0.358574
2,precision,0.611878,0.525508,0.482462,0.391440,0.568360,0.528515,0.499030
3,recall,0.532828,0.560606,0.497475,0.530303,0.491525,0.515738,0.452785
4,f1,0.533459,0.539986,0.483438,0.442092,0.503232,0.517810,0.468559


#### SVM

In [93]:
base_rf = SVC(random_state=1203)
chain = ClassifierChain(base_rf, order='random', random_state=1203)

y_pred=chain.fit(X_train, Y_train).predict(X_test)
#y_prob=chain.predict_proba(X_test)


# Predictions
Y_pred_train = chain.predict(X_train)
#Y_pred_test = chain.predict(X_test)

# Jaccard Score
train_jaccard = jaccard_score(Y_train, Y_pred_train, average="samples")
test_jaccard = jaccard_score(Y_test, y_pred, average="samples")

print(f"Train Jaccard: {train_jaccard}")
print(f"Test Jaccard:  {test_jaccard}")



metrics_df["CC_topSVM"] = [train_jaccard, test_jaccard,
                    precision_score(Y_test, y_pred,average="weighted"), #'precision'
                    recall_score(Y_test, y_pred,average="weighted"), #'recall'
                    f1_score(Y_test, y_pred,average="weighted")]

metrics_df

Train Jaccard: 0.43367428800641794
Test Jaccard:  0.4426282051282051


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,metrics,CC_RF,CC_DT,CC_KNN,CC_SVM,CC_topRF,CC_topDT,CC_topKNN,CC_topSVM
0,train_jacc,0.837906,0.851404,0.545367,0.437706,0.906298,0.891496,0.532371,0.433674
1,test_jacc,0.436378,0.393510,0.371554,0.438381,0.418109,0.386538,0.358574,0.442628
2,precision,0.611878,0.525508,0.482462,0.391440,0.568360,0.528515,0.499030,0.369999
3,recall,0.532828,0.560606,0.497475,0.530303,0.491525,0.515738,0.452785,0.520581
4,f1,0.533459,0.539986,0.483438,0.442092,0.503232,0.517810,0.468559,0.430815


most overfitting in DT & RF

no overfitting in SVM (again)

This feature selection technique does not seem to significantly improve the models, but also gives similar results (successful dimensionality reduction?)


